# SPELL CHECKER APPLICATION

## This is a project to show how to implement spelling checker

## How it works? Some probability theory

The call correction(w) tries to choose the most likely spelling correction for w. There is no way to know for sure (for example, should "lates" be corrected to "late" or "latest" or "lattes" or ...?), which suggests we use probabilities. We are trying to find the correction c, out of all possible candidate corrections, that maximizes the probability that c is the intended correction, given the original word w:

    argmaxc ∈ candidates P(c|w) 

By Bayes' Theorem this is equivalent to:

    argmaxc ∈ candidates P(c) P(w|c) / P(w) 

Since P(w) is the same for every possible candidate c, we can factor it out, giving:

    argmaxc ∈ candidates P(c) P(w|c) 

The four parts of this expression are:

    Selection Mechanism: argmax
    We choose the candidate with the highest combined probability.

    Candidate Model: c ∈ candidates
    This tells us which candidate corrections, c, to consider.

    Language Model: P(c)
    The probability that c appears as a word of English text. For example, occurrences of "the" make up about 7% of English text, so we should have P(the) = 0.07.

    Error Model: P(w|c)
    The probability that w would be typed in a text when the author meant c. For example, P(teh|the) is relatively high, but P(theeexyz|the) would be very low. 

Now let us start with some code.

In [10]:
import re
from collections import Counter
import numpy as np
from sets import Set

/home/vagisha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: DeprecationWarning: the sets module is deprecated


## Generating error matrices

For the error model, we need to generate error matrix. 
We'll have insert matrix, delete matrix etc.

For example : insertMatrix['a']['e'] means how many times from correct word to incorrect word, we have inserted an 'e' after an 'a'

For generating the matrix, we need a list of common errors. Here I have used Peter Norwig's list of errors

In [141]:
file_new = open('errors.txt')

In [142]:
ERRORS = file_new.read()

In [143]:
len(ERRORS.split("\n"))

7843

In [144]:
ERRORS = ERRORS.lower()

In [148]:
x = ERRORS.split("\n")

The following shows list of first 5 in Peter Norwig's list of errors. 
raining is erroneously written as rainning and raning

In [150]:
x[:5]

['raining: rainning, raning',
 'writings: writtings',
 'disparagingly: disparingly',
 'yellow: yello',
 'four: forer, fours, fuore, fore*5, for*4']

In [48]:
left = []
right = []

Here I split the errors into the correct word and the list of incorrect words

In [49]:
count = 0
for i in x :
    spl = i.split(':',1)
    if len(spl)==2 :
        left.append(spl[0])
        right.append(spl[1])
    count = count +1

In [51]:
left[0:10]

['raining',
 'writings',
 'disparagingly',
 'yellow',
 'four',
 'woods',
 'hanging',
 'aggression',
 'looking',
 'eligible']

In [52]:
right[0:10]

[' rainning, raning',
 ' writtings',
 ' disparingly',
 ' yello',
 ' forer, fours, fuore, fore*5, for*4',
 ' woodes',
 ' haing',
 ' agression',
 ' loking, begining, luing, look*2, locking, lucking, louk, looing, lookin, liking',
 ' eligble, elegable, eligable']

Define the error matrices

In [54]:
delMatrix = [[0]*26 for i in range(26)]
insMatrix = [[0]*26 for i in range(26)]
subMatrix = [[0]*26 for i in range(26)]
excMatrix = [[0]*26 for i in range(26)]

In [35]:
chars = ["a", "b", "c", "d", "e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]


In [57]:
def populate_ins(x,y) :
    index_x = chars.index(x)
    index_y = chars.index(y)
    insMatrix[index_x][index_y] = insMatrix[index_x][index_y]+1

In [58]:
def populate_del(x,y) :
    index_x = chars.index(x)
    index_y = chars.index(y)
    #print(x,y)
    delMatrix[index_x][index_y] = delMatrix[index_x][index_y]+1

In [59]:
def populate_sub(x,y) :
    index_x = chars.index(x)
    index_y = chars.index(y)
    subMatrix[index_x][index_y] = subMatrix[index_x][index_y]+1

In [60]:
def populate_exc(x,y) :
    index_x = chars.index(x)
    index_y =chars.index(y)
    excMatrix[index_x][index_y] = excMatrix[index_x][index_y]+1

The populateConfusionMatrix(word,errword) is the module to fill the error matrices given a word and it's errored word. It generates the edit distance matrix and the backtracks along the matrix to find what type of error has occurred.

In [151]:
def populateConfusionMatrix(word,errword):
    dp = [[0]*(len(errword)+1) for i in range(len(word)+1)]
    m = len(word)+1;
    n = len(errword)+1;
    for i in range(m):
        for j in range(n):
            dp[i][0] = i;
            dp[0][j] = j;
    for i in range(m):
        for j in range(n): 
            #print(i,j)
            if i==0 or j==0 :
                continue
                
            dis = [0]*4
            dis[0] = dp[i-1][j]+1
            dis[1] = dp[i][j-1]+1
            #print("dis[1] : " ,dp[i][j-1]+1)
            if word[i-1] == errword[j-1]:
                dis[2] = dp[i-1][j-1]
            else :
                dis[2] = dp[i-1][j-1]+1
                
            if i>1 and j>1 and word[i-1] == errword[j-2] and word[i-2] == errword[j-1]:
                dis[3] = dp[i-2][j-2] + 1 
            if dis[3]!=0 :
                dp[i][j] = min(dis[0:4])
            else :
                dp[i][j] = min(dis[0:3])
               
    i = m-1
    j = n-1
    while(i>0 and j>0) :
        #print("in while loop")
        if word[i-1] == errword[j-1] :
            i=i-1
            j=j-1
            continue
        if dp[i][j] == dp[i][j-1]+1 :
            populate_ins(word[i-1],errword[j-1])
            j=j-1
        if dp[i][j] == dp[i-1][j]+1 :
            populate_del(errword[j-1],word[i-1])
            i=i-1
        if dp[i][j] == dp[i-1][j-1] + 1 :
            populate_sub(word[i-1],errword[j-1])
            i=i-1
            j=j-1
        if i>1 and j>1 and word[i-1] == errword[j-2] and word[i-2] == errword[j-1] and dp[i][j] == dp[i-2][j-2]+1 :
            populate_exc(word[i-2],word[i-1])
            i=i-2
            j=j-2
                

The following is the main module to fill matrices based on all errors in the given list. 

In [62]:
def errorPopulate():
    symbol = "abcdefghijklmnopqrstuvwxyz"
    star = "*"
    size = len(left)
    for i in range(size):
        flag = 0
        word = left[i]
        for x in word :
            if x not in symbol :
                flag = 1
                break
        if flag==1 :
            continue
        numErrorWords = len(right[i].split(","))
        allErrWords = right[i].split(",")
        for j in range(numErrorWords) :
            errorWord = allErrWords[j]
            for k in range(len(errorWord)):
                if errorWord[k] == " ":
                    continue
                else :
                    break        
            errorWord = errorWord[k:]   
            flag2 =0
            for x in errorWord :
                if x not in symbol :
                    flag2 = 1
                    break
            if flag2==1 :
                continue
            print("\n"+errorWord)
            size = 1
            for k in range(len(errorWord)) :
                if errorWord[k] == "*" :
                    size = int(errorWord[k+1:])
                    errorWord = errorWord[0:k]
                    break
                else :
                    continue
            while size>0 :
                print("Populating "+ word+ "and"+ errorWord)
                populateConfusionMatrix(word,errorWord)
                print("Complete")
                size = size-1
            
        

Now these matrices are to be saved because these computations need not be done everytime

In [70]:
np.savetxt("insertMatrix",insMatrix)
np.savetxt("deleteMatrix",delMatrix)
np.savetxt("substituteMatrix",subMatrix)
np.savetxt("exchangeMatrix",excMatrix)

## Data : Text and Words

We need some words to generate the language model.
The list WORDS is a list of the words in the TEXT, but it can also serve as a generative model of text. We know that language is very complicated, but we can create a simplified model of language that captures part of the complexity. In the bag of words model, we ignore the order of words, but maintain their frequency.

Language Model: P(c)
The probability that c appears as a word of English text. For example, occurrences of "the" make up about 7% of English text, so we should have P(the) = 0.07.

First we need some text, possibly from a file. Then we can break the text into words. Here I am using Peter Norwig's big.txt

In [40]:
file_new = open('words.txt')

In [41]:
TEXT = file_new.read()

The following module breaks the given text into words

In [42]:
def tokenize(text):
    return re.findall('[a-z]+',text.lower())

How many words do we have? It's more than a million words

In [43]:
len(tokenize(TEXT))

1105285

In [44]:
WORDS = tokenize(TEXT)

In [45]:
print(WORDS[:10])

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes']


One representation for a bag of words is a Counter, which is a dictionary of {'word': count} pairs. For example,

In [152]:
Counter(tokenize('Is this a test? It is a test!'))

Counter({'a': 2, 'is': 2, 'it': 1, 'test': 2, 'this': 1})

In [46]:
count = Counter(WORDS)

In [78]:
count.most_common(15)

[('the', 80030),
 ('of', 40025),
 ('and', 38313),
 ('to', 28766),
 ('in', 22050),
 ('a', 21155),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681),
 ('his', 10034),
 ('is', 9774),
 ('with', 9740),
 ('as', 8064),
 ('i', 7687)]

## Generating Candidates

For now, we are only considering words which are 0 or 1 edit distance away.

In [31]:
def splits(word) :
    return [(word[:i],word[i:]) for i in range(len(word)+1)]

The splits(word) function splits up the word into two at every location. 

In [32]:
print(splits("wird"))

[('', 'wird'), ('w', 'ird'), ('wi', 'rd'), ('wir', 'd'), ('wird', '')]


This generates all 1-edit distance away words frome the given words, the 1-edit distance being achievable from either insert, delete, replace or transpose.

In [33]:
def edits1(word) :
    pairs = splits(word)
    inserts= [a+c+b for (a,b) in pairs for c in chars]
    deletes= [a+b[1:] for (a,b) in pairs if len(b)>0]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in chars if len(b)>0]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b)>1]
    
    return set(inserts+deletes+replaces+transposes)

In [36]:
print(len(edits1("wird")))

234


In [70]:
def edits0(word):
    return set([word])

Now, every 1-edit distance away word is not useful. We need only those words which make sense, or are in the given dictionary. Therefore, here we generate a subset of these words by checking if they are in our bag of words.

In [71]:
def inDictionary(words) :
    return {w for w in words if w in count}

In [72]:
print(inDictionary(edits1("wird")))

set(['wire', 'word', 'sird', 'wirt', 'gird', 'wid', 'wired', 'wild', 'weird', 'ward', 'bird', 'wind', 'wiry'])


## Probabilistic Language Model

The language model we are using is basically just the probability of the word in the given text.
So, the module given below returns this probability.

In [77]:
def probLangModel(word) :
    N = sum(count.values())
    return float(float(count[word])/N)


In [108]:
probLangModel("wild")

3.16660408853825e-05

In [103]:
for w in tokenize('"The" is most common word in English'):
    print probLangModel(w), w

0.0724066643445 the
0.00884296810325 is
0.000821507574969 most
0.00025966153526 common
0.000269613719538 word
0.0199496057578 in
0.000190900989338 english


## Probabilistic Error Model

First, we load the error matrices generated above.

In [11]:
insArray = np.loadtxt("insertMatrix")
delArray = np.loadtxt("deleteMatrix")
subArray = np.loadtxt("substituteMatrix")
excArray = np.loadtxt("exchangeMatrix")

It is important to find what type of error has occurred and where. For example if "ready" is written as "read", this module will tell us that a deletion has occurred and that a 'y' has been deleted after a 'd'.

In [135]:
def errorType(word,errword):
    if word == errword :
        return "no-op"
    dp = [[0]*(len(errword)+1) for i in range(len(word)+1)]
    m = len(word)+1;
    n = len(errword)+1;
    for i in range(m):
        for j in range(n):
            dp[i][0] = i;
            dp[0][j] = j;
    for i in range(m):
        for j in range(n): 
            #print(i,j)
            if i==0 or j==0 :
                continue
                
            dis = [0]*4
            dis[0] = dp[i-1][j]+1
            dis[1] = dp[i][j-1]+1
            #print("dis[1] : " ,dp[i][j-1]+1)
            if word[i-1] == errword[j-1]:
                dis[2] = dp[i-1][j-1]
            else :
                dis[2] = dp[i-1][j-1]+1
                
            if i>1 and j>1 and word[i-1] == errword[j-2] and word[i-2] == errword[j-1]:
                dis[3] = dp[i-2][j-2] + 1 
            if dis[3]!=0 :
                dp[i][j] = min(dis[0:4])
            else :
                dp[i][j] = min(dis[0:3])
                
    if dp[m-1][n-1] == 1 :
        #return 1 operation
        i = m-1
        j = n-1
        while(i>0 and j>0) :
        #print("in while loop")
            if word[i-1] == errword[j-1] :
                i=i-1
                j=j-1
                continue
            if dp[i][j] == dp[i][j-1]+1 :
                return "insert:"+word[i-1]+":"+errword[j-1]
                #continue insert
            if dp[i][j] == dp[i-1][j]+1 :
            #print(i,j) delete
                return "delete:"+errword[j-1]+":"+word[i-1]
            #continue
            if dp[i][j] == dp[i-1][j-1] + 1 :
                return "sub:"+word[i-1]+":"+errword[j-1]
            #continue
            if i>1 and j>1 and word[i-1] == errword[j-2] and word[i-2] == errword[j-1] and dp[i][j] == dp[i-2][j-2]+1 :
                return "exchange:"+word[i-1]+":"+word[i-2]
               

An example for the above function :

In [153]:
errorType("ready","read")

'delete:d:y'

The following function first determines the error type and then finds the value of the error matrix on the type and error occured. For example : "ready" to "read" will find the count of delArray['d']['y'].

In [154]:
def errorCount(x,y) :
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    operation = errorType(x,y)
    if operation == "no-op" :
        return 1.0
    else :
        text = operation.split(":")
        if text[0] == "insert" :
            #insert probability
            index1 = alphabet.index(text[1])
            index2 = alphabet.index(text[2])
            return insArray[index1][index2]
        elif text[0] == "delete" :
            #delete probability
            index1 = alphabet.index(text[1])
            index2 = alphabet.index(text[2])
            return delArray[index1][index2]
        elif text[0] == "sub" :
            #substitute probability
            index1 = alphabet.index(text[1])
            index2 = alphabet.index(text[2])
            return subArray[index1][index2]
        elif text[0] == "exchange" :
            #exchange probability
            index1 = alphabet.index(text[1])
            index2 = alphabet.index(text[2])
            return excArray[index1][index2]
    
        

Now, finally we need to find the probability of the error occurred. For this, we will divide the count generated from the above function by the total count.

In [110]:
def probErrorModel(x,y):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    operation = errorType(x,y)
    #count = errorCount(x,y)
    #print(count)
    
    if operation == "no-op" :
        return 1.0
    elif operation is None :
        return 0.000001
    else:

        text = operation.split(":")
        count = errorCount(x,y)
        index = alphabet.index(text[1])
        sum = 0
        for i in range(26):
            sum += insArray[index][i]
            sum += delArray[index][i]
            sum += subArray[index][i]
            sum += excArray[index][i]
         
        #print(sum)
        return count/sum   

        
        

Probability of error, writing "read" instead of "ready" is :

In [155]:
probErrorModel("ready","read")

0.010299166257969592

## Combining Error Model and Language Model and generating a list of top candidates for the given erroneous word

Since P(w) is the same for every possible candidate c, we can factor it out, giving:

argmaxc ∈ candidates P(c) P(w|c) 

Thus, multiplying both the probabilities, and sorting in decreasing order, we can find the top candidate given by the model.

In [156]:
def getkey(keys) :
    return keys[1]

def gencorrectWords(word) :
    relatedWords = inDictionary(edits0(word))|inDictionary(edits1(word))
    #print(relatedWords)
    listOfWords = []
    for w in relatedWords:
        #print(word,w)
        probability = probErrorModel(w,word)*probLangModel(w)
        listOfWords.append((w,probability))
    #print(listOfWords)
    listOfWords = sorted(listOfWords,key=getkey,reverse=True)
    return listOfWords

Let's check this out, by giving the erroneous word "maet" , we find the top candidate to be "meet" by this model.

In [138]:
gencorrectWords("maet")

[('meet', 1.8954854861327568e-05),
 ('met', 7.8550476652382218e-06),
 ('mast', 6.2892591373217227e-08),
 ('malt', 5.7743956908638683e-08),
 ('mat', 4.0691030914396161e-08),
 ('matt', 3.5996263311849448e-08),
 ('mate', 9.208951087422766e-09),
 ('meat', 8.4188339822888612e-09),
 ('aet', 3.6189761011865716e-11)]

This is the simplest implementation of a spell - checker.

There are various issues to be looked into.

1. Instead of using Unigram model, we can use Bigram or Trigram model which could effectively take into account the surrounding words of the given erroneous word.

2. Here if our dictionary does not contain a word, then it's probability of occurence is taken as zero, which is unnecessarily stringent. Our dictionary might miss certain words.We can address this by assigning a non-zero probability to words that are not in the dictionary. This is even more important when it comes to multi-word phrases (such as bigrams), because it is more likely that a legitimate one will appear that has not been observed before.We can think of our model as being overly spiky; it has a spike of probability mass wherever a word or phrase occurs in the corpus. What we would like to do is smooth over those spikes so that we get a model that does not depend on the details of our corpus. The process of "fixing" the model is called smoothing.

These two issues will be addressed in the next section.